In [1]:
# skj_extractor.py
import os
import json
import re
from datetime import datetime
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, TextLoader
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
import sys
import os

# Add current directory to path to import prompt
from prompt.prompt import MANAGERIAL_ASSESSMENT_PROMPT
from prompt.prompt import EXTRACT_SKJ_PROMPT


class SKJExtractor:
    def __init__(self, llm, skj_folder="data/skj"):
        self.llm = llm
        self.skj_folder = Path(skj_folder)
        self.skj_folder.mkdir(parents=True, exist_ok=True)
        self.extraction_chain = LLMChain(llm=llm, prompt=EXTRACT_SKJ_PROMPT)
        
    def load_skj_document(self, file_path):
        """Load SKJ document based on file type"""
        file_path = Path(file_path)
        
        if file_path.suffix.lower() == '.pdf':
            loader = PyPDFLoader(str(file_path))
        elif file_path.suffix.lower() in ['.docx', '.doc']:
            loader = Docx2txtLoader(str(file_path))
        elif file_path.suffix.lower() == '.txt':
            loader = TextLoader(str(file_path), encoding='utf-8')
        else:
            raise ValueError(f"Unsupported file format: {file_path.suffix}")
            
        documents = loader.load()
        full_text = "\n".join([doc.page_content for doc in documents])
        return self.preprocess_text(full_text)
    
    def preprocess_text(self, text):
        """Preprocess SKJ text"""
        # Remove extra whitespaces and normalize
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'-\s*\d+\s*-', '', text)  # Remove page numbers
        return text.strip()
    
    def extract_skj(self, file_path):
        """Extract SKJ data from document"""
        try:
            print(f"Memproses file: {file_path}")
            skj_text = self.load_skj_document(file_path)
            
            # Extract using LLM
            result = self.extraction_chain.invoke({"skj_text": skj_text[:8000]})  # Limit text length
            extracted_data = self.parse_json_output(result['text'])
            
            # Add metadata
            extracted_data['metadata']['sumber_file'] = os.path.basename(file_path)
            extracted_data['metadata']['extracted_at'] = datetime.now().isoformat()
            
            return extracted_data
            
        except Exception as e:
            print(f"Error extracting SKJ from {file_path}: {str(e)}")
            return None
    
    def parse_json_output(self, text):
        """Parse JSON output from LLM"""
        try:
            # Extract JSON from text
            json_match = re.search(r'\{.*\}', text, re.DOTALL)
            if json_match:
                json_str = json_match.group()
                return json.loads(json_str)
            else:
                raise ValueError("No JSON found in LLM output")
        except json.JSONDecodeError as e:
            print(f"JSON parsing error: {e}")
            # Return empty structure
            return self.get_empty_skj_structure()
    
    def get_empty_skj_structure(self):
        """Return empty SKJ structure"""
        return {
            "jabatan": "",
            "kode_jabatan": "",
            "unit_organisasi": "",
            "ringkasan_tugas": "",
            "kompetensi_manajerial": [],
            "kompetensi_teknis": [],
            "persyaratan_jabatan": {
                "pendidikan": "",
                "pelatihan": "",
                "pengalaman": ""
            },
            "metadata": {
                "sumber_file": "",
                "extracted_at": "",
                "extractor_version": "v1"
            }
        }
    
    def get_available_skj_files(self):
        """Get list of available SKJ files"""
        skj_files = []
        for ext in ['*.pdf', '*.docx', '*.doc', '*.txt']:
            skj_files.extend(self.skj_folder.glob(ext))
        return [f.name for f in skj_files]
    
    def load_skj_data(self, filename):
        """Load extracted SKJ data from file"""
        skj_file = self.skj_folder / "extracted" / f"{Path(filename).stem}.json"
        if skj_file.exists():
            with open(skj_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        return None
    
    def save_skj_data(self, skj_data, filename):
        """Save extracted SKJ data"""
        output_dir = self.skj_folder / "extracted"
        output_dir.mkdir(parents=True, exist_ok=True)
        
        output_file = output_dir / f"{Path(filename).stem}.json"
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(skj_data, f, indent=2, ensure_ascii=False)
        
        return output_file

class SKJManager:
    def __init__(self, extractor):
        self.extractor = extractor
        self.selected_skj = None
        
    def list_available_skj(self):
        """List available SKJ files"""
        return self.extractor.get_available_skj_files()
    
    def select_skj(self, filename):
        """Select SKJ for assessment"""
        # Try to load existing extracted data
        skj_data = self.extractor.load_skj_data(filename)
        
        if not skj_data:
            # Extract from original file
            file_path = self.extractor.skj_folder / filename
            if file_path.exists():
                skj_data = self.extractor.extract_skj(file_path)
                if skj_data:
                    self.extractor.save_skj_data(skj_data, filename)
        
        self.selected_skj = skj_data
        return skj_data
    
    def get_selected_skj_context(self):
        """Get context from selected SKJ for RAG"""
        if not self.selected_skj:
            return ""
        
        context_parts = []
        
        # Add basic job information
        context_parts.append(f"JABATAN: {self.selected_skj.get('jabatan', '')}")
        context_parts.append(f"KODE: {self.selected_skj.get('kode_jabatan', '')}")
        context_parts.append(f"UNIT: {self.selected_skj.get('unit_organisasi', '')}")
        context_parts.append(f"RINGKASAN TUGAS: {self.selected_skj.get('ringkasan_tugas', '')}")
        
        # Add managerial competencies
        if self.selected_skj.get('kompetensi_manajerial'):
            context_parts.append("\nKOMPETENSI MANAJERIAL:")
            for comp in self.selected_skj['kompetensi_manajerial']:
                context_parts.append(f"- {comp.get('nama_kompetensi', '')}: {comp.get('definisi', '')}")
                for level in ['level_1', 'level_2', 'level_3', 'level_4']:
                    if comp.get(level):
                        context_parts.append(f"  {level.upper()}: {comp[level].get('deskripsi', '')}")
                        for indicator in comp[level].get('indikator_perilaku', []):
                            context_parts.append(f"    • {indicator}")
        
        # Add technical competencies
        if self.selected_skj.get('kompetensi_teknis'):
            context_parts.append("\nKOMPETENSI TEKNIS:")
            for comp in self.selected_skj['kompetensi_teknis']:
                context_parts.append(f"- {comp.get('nama_kompetensi', '')}: {comp.get('definisi', '')}")
        
        return "\n".join(context_parts)

In [ ]:
import sys
import os
import re 


from langchain_openai import OpenAIEmbeddings
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders  import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA



from langchain_community.vectorstores import FAISS

In [4]:
LLM_MODEL = "mistralai/mistral-7b-instruct-v0.2"  # demo
from langchain_openai import ChatOpenAI
llm_chat = ChatOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY, 
    model= LLM_MODEL,
    temperature=0.7,
    max_tokens=256,
    streaming=True,
    verbose=True,
)

NameError: name 'BASE_URL' is not defined

In [ ]:
# Di notebook setelah imports
from skj_ekstraktor import SKJExtractor, SKJManager
from soal_generator import SoalGenerator

# Initialize SKJ system - menggunakan folder data/skj_documents
skj_extractor = SKJExtractor(llm_chat, skj_folder="data/skj_documents")
skj_manager = SKJManager(skj_extractor)
soal_generator = SoalGenerator(llm_chat)

# List available SKJ files
available_skj = skj_manager.list_available_skj()
print("Available SKJ files:", available_skj)

# Select SKJ (contoh - bisa dibuat interactive)
selected_skj_data = None
if available_skj:
    selected_skj_name = available_skj[0]  # Pilih pertama, bisa dibuat dropdown
    print(f"\nMemproses SKJ: {selected_skj_name}")
    selected_skj_data = skj_manager.select_skj(selected_skj_name)
    if selected_skj_data:
        print(f"✅ SKJ berhasil diekstrak!")
        print(f"   Jabatan: {selected_skj_data.get('jabatan', 'Unknown')}")
        print(f"   Kode Jabatan: {selected_skj_data.get('kode_jabatan', 'Unknown')}")
        print(f"   Unit Organisasi: {selected_skj_data.get('unit_organisasi', 'Unknown')}")
        print(f"   Kompetensi Manajerial: {len(selected_skj_data.get('kompetensi_manajerial', []))} item")
        print(f"   Kompetensi Teknis: {len(selected_skj_data.get('kompetensi_teknis', []))} item")
    else:
        print("❌ Gagal mengekstrak SKJ")
else:
    print("⚠️ Tidak ada file SKJ ditemukan di data/skj_documents")

NameError: name 'llm_chat' is not defined

### Langgraph docs loader


In [ ]:
from langchain_community.document_loaders  import PyPDFLoader


loader = PyPDFLoader("./data/PERMENPAN NOMOR 38 TAHUN 2017.pdf")
documents = loader.load()

In [ ]:
documents[0]

Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 0, 'page_label': '1'}, page_content='PERATURAN MENTERI PENDAYAGUNAAN APARATUR NEGARA DAN\nREFORMASI BIROKRASI REPUBLIK INDONESIA\nNOMOR 38 TAHUN 2017\nTENTANG\nSTANDAR KOMPETENSI JABATAN APARATUR SIPIL NEGARA\nDENGAN RAHMAT TUHAN YANG MAHA ESA\nMENTERI PENDAYAGUNAAN APARATUR NEGARA\nDAN REFORMASI BIROKRASI REPUBLIK INDONESIA,\nMenimbang : bahwa untuk melaksanakan ketentuan Pasal 55 ayat (5),\nPasal 109 ayat (4) dan ayat (5) dan Pasal 166 ayat (2)\nPeraturan Pemerintah Nomor 11 Tahun 2017 tentang\nManajemen Pegawai Negeri Sipil, perlu menetapkan Peraturan\nMenteri Pendayagunaan Aparatur Negara dan Reformasi\nBirokrasi tentang Standar Kompetensi Jabatan Aparatur Sipil\nNegara;\nMengingat : 1. Undang-Undang Nomor 5 Tahun 2014 tentang Aparatur\nSipil Negara (Lembaran Negara Republik Indonesia Tahun\n2014 Nomor 6, Tambahan Lembaran

In [ ]:
import pprint

pprint.pp(documents[0].metadata)

{'producer': 'PyPDF',
 'creator': 'PyPDF',
 'creationdate': '',
 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf',
 'total_pages': 108,
 'page': 0,
 'page_label': '1'}


### Preprocessing

In [ ]:
import re 
def preprocess(text): 
    # Menghilangkan footer nomor halaman seperti "- 5 -"
    text = re.sub(r'-\s*\d+\s*-', '', text)
    text = re.sub(r'\n\s*\n', '\n\n', text)
    text = re.sub(r'\n(?!\n)', ' ', text)
    text = re.sub(r' +', ' ', text)

    return text.strip().lower()

for doc in documents:
    doc.page_content = preprocess(doc.page_content) 



In [ ]:
documents[0:5]

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 0, 'page_label': '1'}, page_content='peraturan menteri pendayagunaan aparatur negara dan reformasi birokrasi republik indonesia nomor 38 tahun 2017 tentang standar kompetensi jabatan aparatur sipil negara dengan rahmat tuhan yang maha esa menteri pendayagunaan aparatur negara dan reformasi birokrasi republik indonesia, menimbang : bahwa untuk melaksanakan ketentuan pasal 55 ayat (5), pasal 109 ayat (4) dan ayat (5) dan pasal 166 ayat (2) peraturan pemerintah nomor 11 tahun 2017 tentang manajemen pegawai negeri sipil, perlu menetapkan peraturan menteri pendayagunaan aparatur negara dan reformasi birokrasi tentang standar kompetensi jabatan aparatur sipil negara; mengingat : 1. undang-undang nomor 5 tahun 2014 tentang aparatur sipil negara (lembaran negara republik indonesia tahun 2014 nomor 6, tambahan lembaran negara republik

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,    
    chunk_overlap=200,  
    length_function=len
)



docs_split = text_splitter.split_documents(documents)

In [ ]:
docs_split[:10]

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 0, 'page_label': '1'}, page_content='peraturan menteri pendayagunaan aparatur negara dan reformasi birokrasi republik indonesia nomor 38 tahun 2017 tentang standar kompetensi jabatan aparatur sipil negara dengan rahmat tuhan yang maha esa menteri pendayagunaan aparatur negara dan reformasi birokrasi republik indonesia, menimbang : bahwa untuk melaksanakan ketentuan pasal 55 ayat (5), pasal 109 ayat (4) dan ayat (5) dan pasal 166 ayat (2) peraturan pemerintah nomor 11 tahun 2017 tentang manajemen pegawai negeri sipil, perlu menetapkan peraturan menteri pendayagunaan aparatur negara dan reformasi birokrasi tentang standar kompetensi jabatan aparatur sipil negara; mengingat : 1. undang-undang nomor 5 tahun 2014 tentang aparatur sipil negara (lembaran negara republik indonesia tahun 2014 nomor 6, tambahan lembaran negara republik

### Configuration

In [ ]:
# LLM Configuration - Using OpenRouter (reliable cloud LLM service)
LLM_API_BASE = "https://openrouter.ai/api/v1"  # OpenRouter endpoint
LLM_API_KEY = "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"  # OpenRouter API key
LLM_MODEL = "mistralai/mistral-7b-instruct-v0.2"  # demo
MAX_TOKENS = 256
EMBEDDING_API_BASE = "https://openrouter.ai/api/v1"  # OpenRouter endpoint untuk embedding
EMBEDDING_MODEL = "qwen/qwen3-embedding-8b"  # Model embedding Qwen 3

In [ ]:
# BASE_URL = "https://vllm.emyulabs.my.id/v1"
# API_KEY = "sk-local-test"

# client = OpenAI(base_url=BASE_URL,
#                 api_key=API_KEY, 
#                 timeout=30.0,
#                 max_retries=3)

BASE_URL = "https://openrouter.ai/api/v1"
API_KEY = "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"

client = OpenAI(base_url=BASE_URL,
                api_key=API_KEY, 
                timeout=30.0,
                max_retries=3)

### Embedding model

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
EMBEDDING_MODEL = "qwen/qwen3-embedding-8b"  # Model embedding Qwen 3

embedding_model = OpenAIEmbeddings(
    base_url=BASE_URL,
    api_key=API_KEY,
    model=EMBEDDING_MODEL 
)


db = FAISS.from_documents(docs_split, embedding_model)

### Instruct model

In [ ]:
LLM_MODEL = "mistralai/mistral-7b-instruct-v0.2"  # demo
from langchain_openai import ChatOpenAI
llm_chat = ChatOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY, 
    model= LLM_MODEL,
    temperature=0.7,
    max_tokens=256,
    streaming=True,
    verbose=True,
)

### Retrieval - Generation

In [ ]:
retriever = db.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k": 10 }
)

# option = mmr or similarity_with_threshold
# retriever_mmr = db.as_retriever(
#     # Ganti search_type menjadi "mmr"
#     search_type="mmr",
#     search_kwargs={
#         "k": 10,           
#         "fetch_k": 20,   
#         "lambda_mult": 0.5 
#     }
# )

query = """Apa inti dalam PERATURAN MENTERI PENDAYAGUNAAN APARATUR NEGARA
 DAN REFORMASI BIROKRASI TENTANG STANDAR
 KOMPETENSI JABATAN APARATUR SIPIL NEGARA ?"""

retrieved_docs = retriever.get_relevant_documents(query)

print(f"Dokumen terambil dari k :{retrieved_docs}")

C:\Users\erwin\AppData\Local\Temp\ipykernel_11652\1728485176.py:21: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


Dokumen terambil dari k :[Document(id='293f1741-afc2-4b66-9f11-b841d504eeed', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 12, 'page_label': '13'}, page_content='standar kompetensi jabatan yang ditetapkan oleh menteri menjadi standar dalam menyelenggarakan manajemen aparatur sipil negara yang berlaku secara nasional. c. ruang lingkup ruang lingkup penyusunan standar kompetensi yang diatur dalam peraturan menteri ini meliputi: 1. pedoman pembentukan dan tugas tim penyusun standar kompetensi; 2. pedoman dan tata cara penyusunan standar kompetensi jabatan dan persyaratan jabatan; 3. pedoman dan tata cara penetapan standar kompetensi jabatan. 4. pedoman pembentukan dan tugas tim penyusun kamus kompetensi teknis; dan 5. pedoman dan tata cara penyusunan kamus kompetensi teknis; d. pengertian dalam peraturan menteri ini yang dimaksud dengan: 1. standar kompetensi jabatan aparatur sipil 

In [ ]:
# OUTPUT MODEL INSTRUCT "Qwen/Qwen2.5-coder-1.5B-Instruct"
retriever = db.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k": 10 }
)

# option = mmr or similarity_with_threshold
# retriever_mmr = db.as_retriever(
#     # Ganti search_type menjadi "mmr"
#     search_type="mmr",
#     search_kwargs={
#         "k": 10,           
#         "fetch_k": 20,   
#         "lambda_mult": 0.5 
#     }
# )

query = """Apa inti dalam PERATURAN MENTERI PENDAYAGUNAAN APARATUR NEGARA
 DAN REFORMASI BIROKRASI TENTANG STANDAR
 KOMPETENSI JABATAN APARATUR SIPIL NEGARA ?"""

retrieved_docs = retriever.get_relevant_documents(query)

print(f"Dokumen terambil dari k :{retrieved_docs}")

Dokumen terambil dari k :[Document(id='293f1741-afc2-4b66-9f11-b841d504eeed', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 12, 'page_label': '13'}, page_content='standar kompetensi jabatan yang ditetapkan oleh menteri menjadi standar dalam menyelenggarakan manajemen aparatur sipil negara yang berlaku secara nasional. c. ruang lingkup ruang lingkup penyusunan standar kompetensi yang diatur dalam peraturan menteri ini meliputi: 1. pedoman pembentukan dan tugas tim penyusun standar kompetensi; 2. pedoman dan tata cara penyusunan standar kompetensi jabatan dan persyaratan jabatan; 3. pedoman dan tata cara penetapan standar kompetensi jabatan. 4. pedoman pembentukan dan tugas tim penyusun kamus kompetensi teknis; dan 5. pedoman dan tata cara penyusunan kamus kompetensi teknis; d. pengertian dalam peraturan menteri ini yang dimaksud dengan: 1. standar kompetensi jabatan aparatur sipil 

## Prompt
Kumpulan prompt

In [ ]:
from langchain.prompts import PromptTemplate

# ==========================================
# 1. BASIC RAG PROMPTS
# ==========================================
prompt_template = """
Gunakan potongan-potongan konteks berikut untuk menjawab pertanyaan pengguna.
Jawablah secara ringkas dan jelas dalam Bahasa Indonesia.
Jika Anda tidak tahu jawabannya berdasarkan konteks yang diberikan,
katakan saja bahwa Anda tidak dapat menemukan informasinya,
jangan mencoba mengarang jawaban.

KONTEKS:
{context}

PERTANYAAN:
{question}

JAWABAN YANG MEMBANTU:
"""

CUSTOM_PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

BASIC_RAG_PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# ==========================================
# 2. MANAGERIAL ASSESSMENT PROMPT
# ==========================================
MANAGERIAL_ASSESSMENT_PROMPT = PromptTemplate(
    template="""
ANDA ADALAH PENILAI KOMPETENSI MANAJERIAL BERDASARKAN PERMENPAN RB NO. 38 TAHUN 2017.

KONTEKS STANDAR KOMPETENSI:
{context}

DATA TEST CASE:
- Nama: {nama}
- Jabatan: {jabatan}
- Jawaban Test Case: {jawaban}
- Kompetensi yang Dinilai: {kompetensi}
- Level Target: {level_target}

INSTRUKSI PENILAIAN:
1. Analisis jawaban berdasarkan indikator perilaku untuk level {level_target}
2. Berikan skor 1-4 (1: Tidak memenuhi, 2: Cukup, 3: Baik, 4: Sangat Baik)
3. Berikan justifikasi berdasarkan standar kompetensi
4. Berikan rekomendasi pengembangan

FORMAT OUTPUT:
SKOR: [angka 1-4]
JUSTIFIKASI: [penjelasan berdasarkan indikator perilaku]
REKOMENDASI: [saran pengembangan kompetensi]

HASIL PENILAIAN:
""",
    input_variables=[
        "context", "nama", "jabatan",
        "jawaban", "kompetensi", "level_target"
    ]
)

# ==========================================
# 3. TECHNICAL ASSESSMENT PROMPT
# ==========================================
TECHNICAL_COMPETENCY_PROMPT = PromptTemplate(
    template="""
ANDA ADALAH PENILAI KOMPETENSI TEKNIS BERDASARKAN PERMENPAN RB NO. 38 TAHUN 2017.

KONTEKS STANDAR KOMPETENSI TEKNIS:
{context}

DATA PENILAIAN:
- Nama: {nama}
- Jabatan: {jabatan}
- Bidang Teknis: {bidang_teknis}
- Jawaban/Karya: {jawaban}
- Level yang Dinilai: {level_target}

INSTRUKSI:
1. Evaluasi berdasarkan standar kompetensi teknis untuk bidang {bidang_teknis}
2. Berikan penilaian kualitatif
3. Identifikasi kekuatan dan kelemahan
4. Rekomendasikan pengembangan teknis

HASIL EVALUASI TEKNIS:
""",
    input_variables=[
        "context", "nama", "jabatan",
        "bidang_teknis", "jawaban", "level_target"
    ]
)

# ==========================================
# 4. GAP ANALYSIS PROMPT
# ==========================================
COMPETENCY_GAP_ANALYSIS_PROMPT = PromptTemplate(
    template="""
ANDA ADALAH ANALIS KOMPETENSI ASN BERDASARKAN PERMENPAN RB NO. 38 TAHUN 2017.

KONTEKS STANDAR KOMPETENSI:
{context}

DATA PROFIL:
- Nama: {nama}
- Jabatan: {jabatan}
- Level Saat Ini: {level_sekarang}
- Level Target: {level_target}
- Hasil Assessment: {hasil_assessment}

INSTRUKSI ANALISIS:
1. Identifikasi gap kompetensi antara level sekarang dan target
2. Rekomendasikan program pengembangan spesifik
3. Saran timeline pengembangan
4. Prioritas kompetensi yang perlu ditingkatkan

FORMAT OUTPUT:
KOMPETENSI MEMADAI: [daftar kompetensi]
KOMPETENSI PERLU DITINGKATKAN: [daftar kompetensi]
PROGRAM PENGEMBANGAN: [rekomendasi program]
TIMELINE: [estimasi timeline]

HASIL ANALISIS:
""",
    input_variables=[
        "context", "nama", "jabatan",
        "level_sekarang", "level_target", "hasil_assessment"
    ]
)

# ==========================================
# 5. EXTRACT SKJ PROMPT (paling penting)
# ==========================================
EXTRACT_SKJ_PROMPT = PromptTemplate(
    template="""
Instruksi:
Anda adalah extractor yang memproses dokumen Standar Kompetensi Jabatan (SKJ).
Tugas Anda: kembalikan **hanya JSON valid**, tanpa teks tambahan, tanpa penjelasan.

Format JSON yang WAJIB dikembalikan:
{
  "jabatan": "",
  "source_file": "",
  "kompetensi":[
    {
      "nama_kompetensi": "",
      "definisi": "",
      "indikator_perilaku": [],
      "contoh_perilaku": [],
      "level_mapping": {
         "1": "",
         "2": "",
         "3": "",
         "4": ""
      }
    }
  ],
  "metadata": {
     "extracted_at": "",
     "extractor_version": "v1"
  }
}

Instruksi tambahan:
- Ambil hanya informasi yang ada dalam dokumen
- Jika tidak ada, isi "" atau []
- Kembalikan output **HANYA JSON**
""",
    input_variables=["nama_file", "teks_dokumen"]
)

# ==========================================
# 6. GENERATE SOAL SKJ PROMPT
# ==========================================
CREATE_SOAL_SKJ_PROMPT = PromptTemplate(
    template="""
Instruksi:
Berdasarkan objek JSON SKJ berikut, buat:
- 5 soal multiple choice per kompetensi
- 3 soal essay per kompetensi

Output: JSON Array.

Format setiap soal:
{
  "id_soal": "",
  "tipe": "mcq" / "essay",
  "soal": "",
  "pilihan": ["A","B","C","D"],
  "jawaban_benar": "",
  "kunci_penilaian": "",
  "kompetensi_target": "",
  "level_target": ""
}

SKJ JSON:
{skj_json}
""",
    input_variables=["skj_json"]
)

print("🎉 Semua prompt berhasil di-load dalam notebook!")


🎉 Semua prompt berhasil di-load dalam notebook!


In [ ]:
# #Prompt
# from langchain.prompts import PromptTemplate
# from pathlib import Path

# Path("prompt").mkdir(exist_ok=True)

# # Template ini akan diisi dengan dokumen dari retriever dan pertanyaan dari query
# prompt_template = """
# Gunakan potongan-potongan konteks berikut untuk menjawab pertanyaan pengguna.
# Jawablah secara ringkas dan jelas dalam Bahasa Indonesia.
# Jika Anda tidak tahu jawabannya berdasarkan konteks yang diberikan, katakan saja bahwa Anda tidak dapat menemukan informasinya, jangan mencoba mengarang jawaban.

# KONTEKS:
# {context}

# PERTANYAAN:
# {question}

# JAWABAN YANG MEMBANTU:
# """

# CUSTOM_PROMPT = PromptTemplate(
#     template=prompt_template, input_variables=["context", "question"]
# )

In [ ]:
from langchain.chains import RetrievalQA
rag_chain = RetrievalQA.from_chain_type(
    llm = llm_chat,
    chain_type = "stuff", 
    retriever = retriever,
    chain_type_kwargs = {"prompt": CUSTOM_PROMPT},
    return_source_documents=True
)

In [ ]:
# Menjalankan query melalui RAG Chain
response = rag_chain.invoke(query)

print("\nPERTANYAAN:")
print(query)

print("\nJAWABAN YANG DIHASILKAN LLM:")
print(response['result'])

print("\nDOCUMENT:")
print(response['source_documents'])


retrieved_docs


PERTANYAAN:
Apa inti dalam PERATURAN MENTERI PENDAYAGUNAAN APARATUR NEGARA
 DAN REFORMASI BIROKRASI TENTANG STANDAR
 KOMPETENSI JABATAN APARATUR SIPIL NEGARA ?

JAWABAN YANG DIHASILKAN LLM:
 Inti dalam Peraturan Menteri Pendayagunaan Aparatur Negara dan Reformasi Birokrasi tentang Standar Kompetensi Jabatan Aparatur Sipil Negara adalah mengatur dan mendefinisikan standar kompetensi yang diperlukan oleh aparatur sipil negara dalam melaksanakan tugas-tugasnya secara profesional dengan pedoman yang ada, termasuk dalam bidang teknis, sosial-kultural, dan pengambilan keputusan. Ini bertujuan untuk membuat aparatur sipil negara lebih efisien, efektif, dan profesional dalam melakukan kegiatan pemerintahan, pembangunan, dan pelayanan publik.

DOCUMENT:
[Document(id='293f1741-afc2-4b66-9f11-b841d504eeed', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 12, 'page_label': '13'}, page_content=

[Document(id='293f1741-afc2-4b66-9f11-b841d504eeed', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 12, 'page_label': '13'}, page_content='standar kompetensi jabatan yang ditetapkan oleh menteri menjadi standar dalam menyelenggarakan manajemen aparatur sipil negara yang berlaku secara nasional. c. ruang lingkup ruang lingkup penyusunan standar kompetensi yang diatur dalam peraturan menteri ini meliputi: 1. pedoman pembentukan dan tugas tim penyusun standar kompetensi; 2. pedoman dan tata cara penyusunan standar kompetensi jabatan dan persyaratan jabatan; 3. pedoman dan tata cara penetapan standar kompetensi jabatan. 4. pedoman pembentukan dan tugas tim penyusun kamus kompetensi teknis; dan 5. pedoman dan tata cara penyusunan kamus kompetensi teknis; d. pengertian dalam peraturan menteri ini yang dimaksud dengan: 1. standar kompetensi jabatan aparatur sipil negara yang selanjutnya d

### Rag untuk penilaian


In [ ]:
llm_assessor = ChatOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY,
    model = LLM_MODEL,
    temperature=0.3, 
    max_tokens=512,
    streaming=False,
    verbose=True,
)

In [ ]:
from prompt.prompt import MANAGERIAL_ASSESSMENT_PROMPT
from langchain.chains import LLMChain


# LLM CHAIN 
assesment_chain =  LLMChain(
    llm = llm_assessor, 
    prompt= MANAGERIAL_ASSESSMENT_PROMPT
)


def competency_asessment(nama, jabatan, jawaban_test, kompetensi, level_target, skj_manager=None):
    """
    Fungsi penilaian kompetensi yang menggunakan SKJ terpilih dan PERMENPAN
    
    Args:
        nama: Nama peserta
        jabatan: Jabatan peserta
        jawaban_test: Jawaban test case
        kompetensi: Nama kompetensi yang dinilai
        level_target: Level target kompetensi (1-4)
        skj_manager: SKJManager instance (optional, jika None akan menggunakan selected_skj_data)
    """
    # Ambil context dari PERMENPAN
    query = f"kompetensi {kompetensi} level {level_target} indikator perilaku"
    relevant_docs = retriever.get_relevant_documents(query)
    context_permenpan = "\n".join([doc.page_content for doc in relevant_docs])
    
    # Ambil context dari SKJ terpilih
    context_skj = ""
    skj_terpilih = "Tidak ada SKJ terpilih"
    
    if skj_manager and skj_manager.selected_skj:
        context_skj = skj_manager.get_selected_skj_context()
        skj_terpilih = f"{skj_manager.selected_skj.get('jabatan', 'Unknown')} - {skj_manager.selected_skj.get('kode_jabatan', 'Unknown')}"
    else:
        # Coba ambil dari global variable jika ada
        try:
            if 'selected_skj_data' in globals() and selected_skj_data:
                # Buat context dari selected_skj_data
                context_parts = []
                context_parts.append(f"JABATAN: {selected_skj_data.get('jabatan', '')}")
                context_parts.append(f"KODE: {selected_skj_data.get('kode_jabatan', '')}")
                context_parts.append(f"UNIT: {selected_skj_data.get('unit_organisasi', '')}")
                context_parts.append(f"RINGKASAN TUGAS: {selected_skj_data.get('ringkasan_tugas', '')}")
                
                if selected_skj_data.get('kompetensi_manajerial'):
                    context_parts.append("\nKOMPETENSI MANAJERIAL:")
                    for comp in selected_skj_data['kompetensi_manajerial']:
                        context_parts.append(f"- {comp.get('nama_kompetensi', '')}: {comp.get('definisi', '')}")
                        for level in ['level_1', 'level_2', 'level_3', 'level_4']:
                            if comp.get(level):
                                context_parts.append(f"  {level.upper()}: {comp[level].get('deskripsi', '')}")
                                for indicator in comp[level].get('indikator_perilaku', []):
                                    context_parts.append(f"    • {indicator}")
                
                context_skj = "\n".join(context_parts)
                skj_terpilih = f"{selected_skj_data.get('jabatan', 'Unknown')} - {selected_skj_data.get('kode_jabatan', 'Unknown')}"
        except:
            pass  # Jika gagal, gunakan default
    
    # Invoke assessment chain dengan semua parameter yang diperlukan
    assesment_chain_result = assesment_chain.invoke({
        "context_permenpan": context_permenpan,
        "context_skj": context_skj,
        "nama": nama,
        "jabatan": jabatan,
        "jawaban": jawaban_test,
        "kompetensi": kompetensi,
        "level_target": level_target,
        "skj_terpilih": skj_terpilih
    })

    return {
        "hasil": assesment_chain_result['text'],
        "sumber": relevant_docs,
        "skj_terpilih": skj_terpilih
    }
    


In [ ]:
test_cases = [
    {
        "nama": "Budi Santoso",
        "jabatan": "Kepala Seksi",
        "jawaban_test": """
        Dalam menyelesaikan konflik tim, saya pertama-tama mengumpulkan semua pihak 
        untuk mendengarkan perspektif masing-masing. Saya menggunakan pendekatan win-win solution 
        dengan mencari titik temu yang menguntungkan semua pihak. Setelah mencapai kesepakatan, 
        saya memastikan implementasinya dengan monitoring berkala dan evaluasi hasil.
        """,
        "kompetensi": "Kerjasama",
        "level_target": 3
    },
    {
        "nama": "Siti Rahayu",
        "jabatan": "Analis Kepegawaian",
        "kompetensi": "Manajemen SDM", 
        "jawaban_test": """
        Untuk menyusun perencanaan SDM, saya melakukan analisis kebutuhan berdasarkan 
        rencana strategis organisasi. Saya menggunakan data historis dan proyeksi kebutuhan 
        untuk menentukan jumlah dan kualifikasi pegawai yang dibutuhkan. 
        Saya juga mempertimbangkan aspek pengembangan karir dan suksesi planning.
        """,
        "level_target": 2
    },
    {
        "nama": "Ani Suryani",
        "jabatan": "Inspektur Madya",
        "jawaban_test": """
        Ketika menemukan adanya potensi penyalahgunaan wewenang di laporan, saya langsung
        melakukan verifikasi silang terhadap data dan peraturan yang berlaku tanpa memandang
        siapa yang terlibat. Saya memastikan semua temuan didukung oleh bukti yang kuat
        dan melaporkannya sesuai prosedur meskipun ada tekanan dari berbagai pihak.
        """,
        "kompetensi": "Integritas",
        "level_target": 4
    }
]


for i, person in enumerate (test_cases, 1): 
    print(f"Penilaian ke {i} dengan nama {person["nama"]}")
    print(f"Jabatan {person["jabatan"]}")
    print("="*60)
    
    # Gunakan skj_manager jika tersedia
    hasil_penilaian = competency_asessment(
        nama=person["nama"],
        jabatan=person["jabatan"],
        jawaban_test=person["jawaban_test"],
        kompetensi=person["kompetensi"],
        level_target=person["level_target"],
        skj_manager=skj_manager if 'skj_manager' in globals() else None
    )

    print("Hasil Penilaian")
    print(hasil_penilaian["hasil"])
    
    if "skj_terpilih" in hasil_penilaian:
        print(f"\n📋 SKJ Terpilih: {hasil_penilaian['skj_terpilih']}")

    print("\n DOKUMEN SUMBER:")
    print("-" * 30)
    for doc_num, doc in enumerate(hasil_penilaian['sumber'], 1):
        page_label = doc.metadata.get('page_label', 'N/A')
        print(f"  Sumber #{doc_num} (Halaman: {page_label}):")
        print(f"  \"{doc.page_content[:250].strip()}...\"\n") 
        

Penilaian ke 1 dengan nama Budi Santoso
Jabatan Kepala Seksi
Hasil Penilaian
 SKOR: 4
JUSTIFIKASI: Dalam jawaban tersebut, Budi Santoso menggunakan pendekatan yang objektif dan transparan dalam mengumpulkan semua pihak untuk mendengarkan perspektif masing-masing. Saya menggunakan pendekatan win-win solution dengan mencari titik temu yang menguntungkan semua pihak, yang menunjukkan bahwa ia memahami dan mengaplikasikan prinsip yang diperlukan dalam mengatasi konflik dengan cara yang objektif, netral, tidak memihak, tidak diskriminatif, serta tidak terpengaruh kepentingan pribadi/kelompok/partai politik.

REKOMENDASI: Untuk pengembangan kompetensi kerjasama, Budi Santoso dapat mengikuti pelatihan atau workshop tentang konflik manajemen dan solusi yang efektif dalam mengatasinya. Selain itu, ia juga dapat melakukan praktik praktis dalam mengatasi konflik di lingkungan instansi dengan cara yang objektif dan transparan.

 DOKUMEN SUMBER:
------------------------------
  Sumber #1 (Halaman: 